# setup

In [ ]:
#setup requirements
%pip install -r ./../requirements.txt

In [ ]:
# load openai_api_key from .env file
from dotenv import load_dotenv
import os

load_dotenv()
openai_api_key = os.environ.get('OPENAI_API_KEY')

model="gpt-4"


In [ ]:
# import of langchain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import StrOutputParser

# copy-paste a media release as 'text_input_medienmitteilung'

In [ ]:
text_input_medienmitteilung = """

    # Medienmitteilung

    ## Der Bundesrat verurteilt die Terroranschläge der Hamas gegen Israel und stärkt seine Handlungsfähigkeit
    
    Bern, 11.10.2023 - Der Bundesrat hat sich an seiner Sitzung vom 11. Oktober 2023 mit den jüngsten schockierenden Angriffen befasst, die von der Hamas aus dem Gazastreifen gegen Zivilistinnen und Zivilisten in Israel verübt wurden. Er verurteilt diese terroristischen Anschläge auf das Schärfste und fordert die sofortige Freilassung der von der Hamas festgehaltenen Geiseln. Angesichts dieser Ereignisse muss die Schweiz wirksam und koordiniert handeln können. Der Bundesrat hat deshalb entschieden, die vom Eidgenössischen Departement für auswärtige Angelegenheiten (EDA) eingesetzte Taskforce zu einer Taskforce des Bundesrates zu erweitern. Der Bundesrat ist der Ansicht, dass die Hamas als terroristische Organisation eingestuft werden sollte.

    Der Bundesrat verurteilt die Terrorangriffe, die von der Hamas aus dem Gazastreifen gegen Zivilistinnen und Zivilisten in Israel verübt wurden, auf das Schärfste und erkennt den legitimen Willen Israels zur nationalen Verteidigung und Sicherheit an. Er ruft zur sofortigen Freilassung der von der Hamas als Geiseln genommenen Personen auf. Der Bundesrat fordert ein sofortiges Ende der Gewalt und erinnert daran, dass die Zivilbevölkerung jederzeit zu schützen und das humanitäre Völkerrecht einzuhalten ist. Er drückt seine Solidarität mit der israelischen Bevölkerung aus und bekundet den Familien der Opfer sein tief empfundenes Mitgefühl.

    Um wirksam und koordiniert handeln zu können, hat der Bundesrat entschieden, die vom Eidgenössischen Departement für auswärtige Angelegenheiten (EDA) eingesetzte Taskforce Naher Osten zu einer Taskforce des Bundesrates zu erweitern. Die vom EDA geleitete TFNO wird durch Vertreterinnen und Vertreter des Präsidialdepartements (EDI), des Eidgenössischen Departements für Verteidigung, Bevölkerungsschutz und Sport (VBS), des Eidgenössischen Justiz- und Polizeidepartements (EJPD) und der Bundeskanzlei (BK) verstärkt. Die Taskforce hat den Auftrag, die notwendigen Analysen durchzuführen, um vor dem Hintergrund der Ereignisse in Israel und im Besetzten Palästinensischen Gebiet koordinierte Entscheidungen treffen zu können. Der Bundesrat ist der Ansicht, dass die Hamas als terroristische Organisation eingestuft werden sollte. Er hat die TFNO beauftragt, die rechtlichen Optionen für ein Verbot der Organisation zu prüfen.

    Der Bundesrat misst dem korrekten Einsatz der finanziellen Unterstützung der Schweiz grosse Bedeutung bei. In diesem Zusammenhang hat das EDA in der Vergangenheit die Verwendung der von der Schweiz an NGO im Nahen Osten gewährten Mittel überprüft und wo nötig Massnahmen ergriffen. Er hat derzeit keine Kenntnis davon, dass Schweizer Gelder der Hamas und ihren Aktivitäten zugute gekommen wären. Das EDA wird eine weitere detaillierte Analyse der Finanzströme im Zusammenhang mit dem Kooperationsprogramm im Nahen Osten durchführen.

    Des Weiteren weist der Bundesrat darauf hin, dass dem Schutz der Sicherheit der Schweizer Bürgerinnen und Bürger jüdischen Glaubens sowie der jüdischen Institutionen eine zentrale Bedeutung zukommt.

    Schliesslich steht die Schweiz zur Verfügung, um alle Bemühungen mit dem Ziel einer Deeskalation in der Region zu unterstützen.

    Adresse für Rückfragen

    für weitere Informationen:
    Kommunikation EDA  
"""

# run this ai-machine

In [ ]:
# result_facts: Analyze the following text and identify all important fact
prompt = PromptTemplate.from_template("""
Analyze the following text and 
    identify all important facts.  
    The text is from news.admin.ch and is an 
    official media release: \n {medienmitteilung}\n\n
    a fact analysis in the same language as
    the media release:
""")
runnable = prompt | ChatOpenAI(model=model) | StrOutputParser()
result_facts = runnable.invoke({"medienmitteilung": text_input_medienmitteilung})
print(result_facts)

In [ ]:
# result_top10keywords: Identify 10 keywords from the facts
prompt = PromptTemplate.from_template("""
    Identify 10 keywords that are likely to rank well 
    on Google (Switzerland). 
    The keywords are in the same language as the 
    following text and start with a #.

    Text: 
    {facts}
    
    Examples: #FederalCouncil #Geneva #Nyon #Motorway #A1 #Traffic #Trafficflow #Motorway     
    Your keywords incl. #:
""")
runnable = prompt | ChatOpenAI(model=model) | StrOutputParser()
result_top10keywords = runnable.invoke({"facts": result_facts})
print(result_top10keywords)

In [ ]:
# result_top3keywords: Analyze the following keywords and text. 
prompt = PromptTemplate.from_template("""
    Analyze the following keywords and text. 
    Identify exactly 3 keywords that can rank 
    well on Google (Switzerland). 

    Examples: #FederalCouncil #ChuckNorris #Thailand  (on one line)
   
    ===============
    Keywords: 
    {top10keywords}

    Text: 
    {medienmitteilung}
    ===============
    Your top 3 keywords in the same language as the text/keywords:
""")
runnable = prompt | ChatOpenAI(model=model) | StrOutputParser()
result_top3keywords = runnable.invoke({"medienmitteilung": text_input_medienmitteilung, "top10keywords": result_top10keywords})
print(result_top3keywords)

In [ ]:
# result_tweets: write 3 matching tweets for a specific media release, facts, keywords.
prompt = PromptTemplate.from_template("""
    You are a neutral media spokesperson for the Federal Council 
    and write 3 matching tweets for a specific media release.
    
    Your tweets always contain:
    1. the 2 most important facts
    2. the 3 most important keywords
    3. 4 matching emojis.
    4. the tweet ends with the string [url].

    Rules:
    You answer in the same language as the media release.

    Sources: 
    ===========
    Facts: {facts}
    Top-Keywords: {top3keywords}
    Media release: {medienmitteilung}
    ==============
    Your 3 different tweet suggestions:
""")
runnable = prompt | ChatOpenAI(model=model) | StrOutputParser()
result_tweets = runnable.invoke({"medienmitteilung": text_input_medienmitteilung, "top3keywords": result_top3keywords, "facts": result_facts})
print(result_tweets)

In [ ]:
# result_fragen_20min: from 20min.ch/Wagenknecht and will collect 3 critical questions. 
prompt = PromptTemplate.from_template("""
    You are a reporter (male) for 20min.ch and 
    will collect 10 critical questions. 
    At the end we need the 3 most critical 
    questions.

    You think like Sahra Wagenknecht. 

    It is important that the questions 
    directly to the press release.
    
    Important:
    The questions are critical but not offensive. 
    and refer to the media release, the facts and a top keyword.

    Rules:
    Answer in the same language as the media release.

    Sources: 
    ===========
    Facts: {facts}
    Top-Keywords: {top3keywords}
    Media release: {medienmitteilung}
    ==============
    Your top-3 moust critical questions:
""")
runnable = prompt | ChatOpenAI(model=model) | StrOutputParser()
result_fragen_20min = runnable.invoke({"medienmitteilung": text_input_medienmitteilung, "top3keywords": result_top3keywords, "facts": result_facts})
print(result_fragen_20min)

In [ ]:
# result_fragen_NZZ: from NZZ/Donald Duck and will collect 3 critical questions. 
prompt = PromptTemplate.from_template("""
You are a reporter (female) for the NZZ and will 
    collect 10 critical questions. 
    At the end we need the 3 most critical 
    questions.

    You think like Donald Duck.

    It is important that the questions 
    relate directly to the press release.
    
    This is important:
    The questions are critical, but not insulting. 
    And refer to the press release, the facts and a keyword.

    Rules:
    Answer in the same language as in the press release.

    Sources: 
    ===========
    Facts {facts}
    Keywords: {top3keywords}
    Media Release: {medienmitteilung}
    ==============
    Your top 3 most critical questions:
""")
runnable = prompt | ChatOpenAI(model=model) | StrOutputParser()
result_fragen_nzz = runnable.invoke({"medienmitteilung": text_input_medienmitteilung, "top3keywords": result_top3keywords, "facts": result_facts})
print(result_fragen_nzz)

In [ ]:
# result_zusammenfassung_kind: summary for a 10 year old child 
prompt = PromptTemplate.from_template("""
Create a summary of the following text 
    for a 10 year old child of a media release.

    Important:
    Answer in the same language as the media release.

    Sources: 
    ===========
    Facts: {facts}
    Media release: {medienmitteilung}
    ==============
    Your child-friendly summary:
""")
runnable = prompt | ChatOpenAI(model=model) | StrOutputParser()
result_zusammenfassung_kind =  runnable.invoke({"medienmitteilung": text_input_medienmitteilung, "facts": result_facts})
print(result_zusammenfassung_kind)

In [ ]:
# result_dalle3prompt: DALL-E prompt : black and white comic book panel in style of asterix & obelix in mid-21th 
# copy-paste the result_dalle3prompt into a good image generator like Midjourney or DallE-3 
prompt = PromptTemplate.from_template("""
Create a prompt to generate 
    with DALL-E a matching image.

    Important Roules:
    =================
    1) Your DALL-E Prompt must be in english.
    2) It must include the following texts:
        A)"create a black and white comic book panel"
        B)"comic illustration reminiscent of European adventure comics of the mid-21th century"
        C)"in a informative way." 
        D) [SCENE]
        E) The scene plays in switzerland.
    3) "Include a dramatic sound effect text saying &quot;{top3keywords}&quot;
    =================

    Contextinformation for the [SCENE]: {zusammenfassung_kind}
    ==============
    Generated the [SCENE] and following the rules 1-3!
    ==============
    Your DALL-E Prompt follows rules 1-3:
""")
runnable = prompt | ChatOpenAI(model=model) | StrOutputParser()
result_dalle3prompt =  runnable.invoke({"zusammenfassung_kind": result_zusammenfassung_kind, "top3keywords": result_top3keywords, "facts": result_facts})
print(result_dalle3prompt)

In [ ]:
# link to a DELL-E 2 image (Midjourney or DallE-3 are much better!)
from langchain.utilities.dalle_image_generator import DallEAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["image_desc"],
    template="{image_desc}",
)
chain = LLMChain(llm=llm, prompt=prompt)
image_url = DallEAPIWrapper().run(chain.run(result_dalle3prompt))
image_url